In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, Add, Average
from tensorflow.keras.metrics import Recall,Precision
from sklearn.metrics import f1_score

In [190]:
asos_df = pd.read_csv('/Volumes/ESD-ISO/project_preson/LSTM/LSTM_data/ASOS_data_preprocessed_V2.csv', index_col=0)

In [191]:
# rain_tomorrow 열을 rainy_day 열에서 3시간 이후 값을 가져와 생성
asos_df['rain_tomorrow'] = asos_df['rainy_day'].shift(-12)

In [192]:
asos_df.drop(index=asos_df[asos_df['rain_tomorrow'].isna()].index, inplace=True) # 24시간 뒤에 데이터가 없는 마지막 24개열 삭제

In [193]:
encoder = LabelEncoder()
asos_df.clfmAbbrCd = encoder.fit_transform(asos_df.clfmAbbrCd)

In [194]:
asos_df['tm'] = pd.to_datetime(asos_df['tm'])

In [195]:
base_time = asos_df['tm'].iloc[0]
asos_df['tm'] = (asos_df['tm']-base_time).dt.total_seconds()

In [196]:
# 'group' 컬럼 생성: 24개 단위로 그룹 ID 부여
asos_df['group'] = np.arange(len(asos_df)) // 24

# rain_tomorrow는 별도로 처리
rain_tomorrow_grouped = asos_df.groupby('group')['rain_tomorrow'].apply(list)

# 나머지 컬럼은 24개 단위로 묶기
other_columns_grouped = asos_df.groupby('group')[['tm','hm', 'pv', 'td', 'pa', 'm005Te', 'm01Te', 'm02Te', 'm03Te', 'clfmAbbrCd', 'lcsCh']].apply(lambda x: x.values.tolist())

# 결과 확인
print("rain_tomorrow:")
print(rain_tomorrow_grouped)

print("\nOther columns:")
print(other_columns_grouped)

rain_tomorrow:
group
0       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              ...                        
2528    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2529    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2530    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2531    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2532    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: rain_tomorrow, Length: 2533, dtype: object

Other columns:
group
0       [[0.0, 57.0, 1.9, -15.1, 1016.4, -3.6, 0.6, 1....
1       [[518400.0, 60.0, 1.5, -17.9, 1017.7, -1.9, -0...
2       [[1036800.0, 38.0, 1.1, -21.5, 1011.7, -3.5, -...
3       [[1555200.0, 67.0, 2.7, -10.7, 1011.3, -4.9, -...
4       [[2073600.0, 67.0, 5.9, -0.5, 

In [197]:
y_data = []
for rain_tomorrow in rain_tomorrow_grouped:
    if 1 in rain_tomorrow:
        y_data.append(1)
        print(1)
    else:
        y_data.append(0)
        print(0)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
0
0
1
1
0
1
1
0
0
0
0
0
0
0
0
0
1
0
1
1
1
0
0
1
0
1
0
1
1
0
1
1
0
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
0
0
1
0
1
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
0
0
0
1
0
1
0
1
0
1
0
1
1
1
0
0
0
1
1
1
1
1
1
0
0
1
0
0
1
1
0
1
0
1
1
1
1
0
0
0
0
0
0
0
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
0
1
0
0
1
0
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
0
1
0
1
1
0
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
0
1
0
0
1
1
1
0
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
1
1
0
1
1
1
0
0
1
1
1
1
1
1
0
1
1
0
1
1
1
0
0
1
0
1
1
0
0
1
1
1
1
0
0
0
0
1
1
0
1
1
1
0
0
1
1
1
0
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
0
1
1
0
1
0
1
1
1
0
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
0
0
1
0
0
1
1
1
0
1
0
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
1
1
1
0
1
1
1
1
1
1
0
1
1
0
1
1
0
1
1
1
1
0
0
0
0
1
1
1
0
0
0
1
1
0
1
1
0
1
1
1
1
0
1
0
1
1
1
1
1
0
0
0
1
0
1
1
1
1
0
0
1
0
0
0
0
1
0
1
1
0
0
1
0
1


In [198]:
np.array(y_data).shape, np.vstack(other_columns_grouped.tolist()).shape

((2533,), (60782, 11))

In [199]:
data_X = np.vstack(other_columns_grouped.iloc[:-1].tolist())

In [200]:
data_X[0]

array([ 0.0000e+00,  5.7000e+01,  1.9000e+00, -1.5100e+01,  1.0164e+03,
       -3.6000e+00,  6.0000e-01,  1.3000e+00,  1.3000e+00,  7.3000e+01,
        0.0000e+00])

In [201]:
data_y = np.array(y_data[:-1])

In [202]:
scaler = MinMaxScaler()
data_X = scaler.fit_transform(data_X).reshape(-1,24,11)

In [203]:
data_X[0]

array([[0.00000000e+00, 5.22222222e-01, 3.50404313e-02, 2.33096085e-01,
        7.91869919e-01, 1.30996310e-01, 2.04255319e-01, 2.00000000e-01,
        2.24806202e-01, 9.86486486e-01, 0.00000000e+00],
       [1.52152964e-05, 4.33333333e-01, 2.69541779e-02, 1.97508897e-01,
        7.96747967e-01, 1.01476015e-01, 1.80851064e-01, 1.89873418e-01,
        2.19638243e-01, 9.86486486e-01, 0.00000000e+00],
       [3.04305929e-05, 3.44444444e-01, 4.31266846e-02, 2.65124555e-01,
        7.49593496e-01, 1.95571956e-01, 1.95744681e-01, 1.92405063e-01,
        2.17054264e-01, 9.86486486e-01, 0.00000000e+00],
       [4.56458893e-05, 6.00000000e-01, 6.73854447e-02, 3.41637011e-01,
        7.15447154e-01, 1.86346863e-01, 1.85106383e-01, 1.92405063e-01,
        2.17054264e-01, 8.78378378e-01, 0.00000000e+00],
       [6.08611858e-05, 9.11111111e-01, 9.97304582e-02, 4.18149466e-01,
        6.91056911e-01, 1.84501845e-01, 1.97872340e-01, 1.89873418e-01,
        2.11886305e-01, 6.75675676e-02, 0.00000000e+

In [204]:
train_X = data_X[:2000]
train_y = data_y[:2000]
test_X = data_X[2000:]
test_y = data_y[2000:]

In [205]:
train_X.shape,train_y.shape,test_X.shape,test_y.shape

((2000, 24, 11), (2000,), (532, 24, 11), (532,))

In [206]:
def lstm_model_V1():

    inputs = Input(shape=(24, 11))

    layer_1 = LSTM(units=32, return_sequences=True)(inputs)

    layer_2 = LSTM(units=64, return_sequences=False)(layer_1) 

    D_1 = Dense(units=64, activation='relu')(layer_2)
    D_2 = Dense(units=128, activation='relu')(D_1)
    D_3 = Dense(units=64, activation='relu')(D_2)
    D_4 = Dense(units=32, activation='relu')(D_3)
    D_5 = Dense(units=16, activation='relu')(D_4)
    outputs = Dense(units=1, activation='sigmoid')(D_5)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [207]:
model = lstm_model_V1()
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 24, 11)]          0         
                                                                 
 lstm_8 (LSTM)               (None, 24, 32)            5632      
                                                                 
 lstm_9 (LSTM)               (None, 64)                24832     
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                                 
 dense_25 (Dense)            (None, 128)               8320      
                                                                 
 dense_26 (Dense)            (None, 64)                8256      
                                                                 
 dense_27 (Dense)            (None, 32)                2080

In [208]:
model.fit(train_X, train_y, validation_split=0.2, epochs=100)

Epoch 1/100


2025-01-15 15:17:54.677026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 15:17:54.808919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 15:17:54.911392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 15:17:55.147934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 15:17:55.416719: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


47/50 [===========================>..] - ETA: 0s - loss: 0.6306 - accuracy: 0.6270

2025-01-15 15:17:56.593027: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 15:17:56.646834: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 15:17:56.745034: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 3s 31ms/step - loss: 0.6294 - accuracy: 0.6294 - val_loss: 0.5348 - val_accuracy: 0.7050
Epoch 2/100
50/50 [==============================] - 1s 18ms/step - loss: 0.5763 - accuracy: 0.7138 - val_loss: 0.5925 - val_accuracy: 0.6750
Epoch 3/100
50/50 [==============================] - 1s 17ms/step - loss: 0.5500 - accuracy: 0.7225 - val_loss: 0.4868 - val_accuracy: 0.7825
Epoch 4/100
50/50 [==============================] - 1s 18ms/step - loss: 0.5254 - accuracy: 0.7412 - val_loss: 0.5049 - val_accuracy: 0.7900
Epoch 5/100
50/50 [==============================] - 1s 18ms/step - loss: 0.5139 - accuracy: 0.7400 - val_loss: 0.5498 - val_accuracy: 0.6775
Epoch 6/100
50/50 [==============================] - 1s 17ms/step - loss: 0.5138 - accuracy: 0.7437 - val_loss: 0.4738 - val_accuracy: 0.7800
Epoch 7/100
50/50 [==============================] - 1s 17ms/step - loss: 0.5059 - accuracy: 0.7450 - val_loss: 0.4650 - val_accuracy: 0.7625
Epoch 8/100
50/50 

50/50 [==============================] - 1s 18ms/step - loss: 0.3283 - accuracy: 0.8569 - val_loss: 0.4558 - val_accuracy: 0.7950
Epoch 59/100
50/50 [==============================] - 1s 18ms/step - loss: 0.2857 - accuracy: 0.8788 - val_loss: 0.6644 - val_accuracy: 0.7475
Epoch 60/100
50/50 [==============================] - 1s 18ms/step - loss: 0.2772 - accuracy: 0.8756 - val_loss: 0.5647 - val_accuracy: 0.7800
Epoch 61/100
50/50 [==============================] - 1s 18ms/step - loss: 0.2879 - accuracy: 0.8650 - val_loss: 0.5482 - val_accuracy: 0.7775
Epoch 62/100
50/50 [==============================] - 1s 18ms/step - loss: 0.2874 - accuracy: 0.8731 - val_loss: 0.5272 - val_accuracy: 0.7700
Epoch 63/100
50/50 [==============================] - 1s 18ms/step - loss: 0.2869 - accuracy: 0.8775 - val_loss: 0.6378 - val_accuracy: 0.7600
Epoch 64/100
50/50 [==============================] - 1s 18ms/step - loss: 0.2559 - accuracy: 0.8831 - val_loss: 0.5513 - val_accuracy: 0.7750
Epoch 65/100

In [209]:
loss, accuracy = model.evaluate(test_X,test_y)
print(f"테스트 정확도: {accuracy*100:.2f}%")

17/17 [==============================] - 0s 20ms/step - loss: 0.8757 - accuracy: 0.7744
테스트 정확도: 77.44%


In [210]:
y_hat = model.predict(test_X)

2025-01-15 15:19:25.235349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 15:19:25.280946: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 15:19:25.381473: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 1s 17ms/step


In [211]:
y_hat = y_hat.argmax(axis=1)

In [212]:
f1 = f1_score(test_y, y_hat, average='micro')
print(f"F1 스코어: {f1:.4f}")

F1 스코어: 0.6786


In [235]:
def lstm_model_V2():

    inputs = Input(shape=(24, 11))

    layer_1_tensor = LSTM(units=32, return_sequences=True)(inputs)
    
    layer_2_1 = LSTM(units=32, return_sequences=True)(layer_1_tensor)
    layer_2_2 = LSTM(units=32, return_sequences=True)(inputs)
    second_layer = Add()([layer_2_1, layer_2_2])
    dropout_1 = Dropout(0.2)(second_layer)
    
    layer_3_tensor = LSTM(units=128, return_sequences=True)(dropout_1)
    dropout_2 = Dropout(0.2)(layer_3_tensor)
    
    layer_4_1_tensor = LSTM(units=64, return_sequences=False)(dropout_2)
    layer_4_2_tensor = LSTM(units=64, return_sequences=False)(layer_1_tensor)
    last_layer = Average()([layer_4_1_tensor, layer_4_2_tensor]) 
    dropout_3 = Dropout(0.2)(last_layer)
    
    D_1 = Dense(units=64, activation='relu')(dropout_3)
    D_2 = Dense(units=32, activation='relu')(D_1)
    dropout_4 = Dropout(0.2)(D_2)
    
    outputs = Dense(units=1, activation='sigmoid')(dropout_4)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',Recall(),Precision()])
    return model

In [236]:
model = lstm_model_V2()

model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 24, 11)]     0           []                               
                                                                                                  
 lstm_40 (LSTM)                 (None, 24, 32)       5632        ['input_13[0][0]']               
                                                                                                  
 lstm_41 (LSTM)                 (None, 24, 32)       8320        ['lstm_40[0][0]']                
                                                                                                  
 lstm_42 (LSTM)                 (None, 24, 32)       5632        ['input_13[0][0]']               
                                                                                            

In [237]:
model.fit(train_X, train_y, validation_split=0.2, epochs=50)

Epoch 1/50


2025-01-15 17:38:59.368075: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:38:59.702324: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:38:59.711382: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:38:59.960002: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:38:59.960023: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:39:00.189526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:39:00.338342: I tensorflow/core/grappler/optimizers/cust

49/50 [============================>.] - ETA: 0s - loss: 0.6257 - accuracy: 0.6614 - recall_3: 0.5375 - precision_3: 0.6507

2025-01-15 17:39:05.554153: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:39:05.687347: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:39:05.697146: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:39:05.827626: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:39:05.887577: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:39:06.072226: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:39:06.176861: I tensorflow/core/grappler/optimizers/cust

50/50 [==============================] - 9s 95ms/step - loss: 0.6243 - accuracy: 0.6644 - recall_3: 0.5422 - precision_3: 0.6555 - val_loss: 0.5700 - val_accuracy: 0.7525 - val_recall_3: 0.4790 - val_precision_3: 0.6064
Epoch 2/50
50/50 [==============================] - 2s 46ms/step - loss: 0.5737 - accuracy: 0.7163 - recall_3: 0.7414 - precision_3: 0.6675 - val_loss: 0.5065 - val_accuracy: 0.7875 - val_recall_3: 0.3697 - val_precision_3: 0.8148
Epoch 3/50
50/50 [==============================] - 2s 45ms/step - loss: 0.5469 - accuracy: 0.7262 - recall_3: 0.7593 - precision_3: 0.6753 - val_loss: 0.4891 - val_accuracy: 0.7775 - val_recall_3: 0.2857 - val_precision_3: 0.8947
Epoch 4/50
50/50 [==============================] - 2s 49ms/step - loss: 0.5338 - accuracy: 0.7250 - recall_3: 0.7856 - precision_3: 0.6659 - val_loss: 0.4803 - val_accuracy: 0.7775 - val_recall_3: 0.6134 - val_precision_3: 0.6293
Epoch 5/50
50/50 [==============================] - 2s 46ms/step - loss: 0.5193 - accur

50/50 [==============================] - 2s 47ms/step - loss: 0.4131 - accuracy: 0.7969 - recall_3: 0.7842 - precision_3: 0.7704 - val_loss: 0.4502 - val_accuracy: 0.7850 - val_recall_3: 0.6555 - val_precision_3: 0.6341
Epoch 37/50
50/50 [==============================] - 2s 46ms/step - loss: 0.4151 - accuracy: 0.7912 - recall_3: 0.8050 - precision_3: 0.7510 - val_loss: 0.4749 - val_accuracy: 0.7700 - val_recall_3: 0.6975 - val_precision_3: 0.5971
Epoch 38/50
50/50 [==============================] - 2s 45ms/step - loss: 0.4031 - accuracy: 0.7931 - recall_3: 0.8064 - precision_3: 0.7532 - val_loss: 0.4513 - val_accuracy: 0.7950 - val_recall_3: 0.5882 - val_precision_3: 0.6796
Epoch 39/50
50/50 [==============================] - 2s 48ms/step - loss: 0.4266 - accuracy: 0.7819 - recall_3: 0.7801 - precision_3: 0.7480 - val_loss: 0.5151 - val_accuracy: 0.7425 - val_recall_3: 0.7143 - val_precision_3: 0.5519
Epoch 40/50
50/50 [==============================] - 2s 46ms/step - loss: 0.4157 - a

In [238]:
loss, accuracy,_,_ = model.evaluate(test_X, test_y)
print(f"테스트 정확도: {accuracy*100:.2f}%")

17/17 [==============================] - 1s 52ms/step - loss: 0.5559 - accuracy: 0.7613 - recall_3: 0.5614 - precision_3: 0.6486
테스트 정확도: 76.13%


In [239]:
y_hat = model.predict(test_X)

2025-01-15 17:41:03.708364: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:41:03.814436: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:41:03.821217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:41:03.974997: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:41:04.034335: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:41:04.229607: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-15 17:41:04.372769: I tensorflow/core/grappler/optimizers/cust

17/17 [==============================] - 2s 45ms/step


In [240]:
y_hat = y_hat.argmax(axis=1)

In [241]:
f1 = f1_score(test_y, y_hat, average='micro')
print(f"F1 스코어: {f1:.4f}")

F1 스코어: 0.6786
